In [1]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer, Trainer, TrainingArguments, TextDataset, DataCollatorForLanguageModeling

/home/27a_sul@lab.graphicon.ru/miniconda3/envs/nlp/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = '1'

In [3]:
model_name = "ai-forever/rugpt2large"
model = GPT2LMHeadModel.from_pretrained(model_name)
tokenizer = GPT2Tokenizer.from_pretrained(model_name)

In [4]:
tokenizer.add_special_tokens({'pad_token': '<|pad|>', 'bos_token': '<|startoftext|>', 'eos_token': '<|endoftext|>'})
model.resize_token_embeddings(len(tokenizer))

Embedding(50260, 1280)

In [5]:
def load_dataset(file_path, tokenizer, block_size=256):
    dataset = TextDataset(
        tokenizer=tokenizer,
        file_path=file_path,
        block_size=block_size
    )
    return dataset

In [ ]:
!gdown 18oCseBD3UpD2ode0TQqMHlcLMULNa1cZ

In [6]:
file_path = "aneki.txt"
train_dataset = load_dataset(file_path, tokenizer)

/home/27a_sul@lab.graphicon.ru/miniconda3/envs/nlp/lib/python3.12/site-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


In [7]:
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False
)

In [8]:
training_args = TrainingArguments(
    output_dir="./model_output",       # Куда сохранять модель
    overwrite_output_dir=True,        # Перезаписывать ли директорию
    num_train_epochs=3,               # Количество эпох
    per_device_train_batch_size=4,    # Размер батчей
    gradient_accumulation_steps=8,    # Увеличивает эффективный размер батча
    save_steps=500,                   # Как часто сохранять чекпоинты
    save_total_limit=2,               # Максимум сохраненных чекпоинтов
    logging_dir="./logs",             # Логи
    logging_steps=100,
    learning_rate=5e-5,
    weight_decay=0.01,
    warmup_steps=500,
    fp16=True,                        # Использовать ли смешанную точность
    prediction_loss_only=True
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    data_collator=data_collator
)

trainer.train()

Step,Training Loss
100,3.488100
200,3.064100
300,3.026300
400,2.996700
500,2.917600
600,2.838100
700,2.842200
800,2.825900
900,2.817500
1000,2.577200


TrainOutput(global_step=1353, training_loss=2.8445280177982664, metrics={'train_runtime': 2670.7288, 'train_samples_per_second': 16.232, 'train_steps_per_second': 0.507, 'total_flos': 4.71054907342848e+16, 'train_loss': 2.8445280177982664, 'epoch': 2.9958483254912815})

In [9]:
trainer.save_model("./fine_tuned_rugpt2")
tokenizer.save_pretrained("./fine_tuned_rugpt2")

('./fine_tuned_rugpt2/tokenizer_config.json',
 './fine_tuned_rugpt2/special_tokens_map.json',
 './fine_tuned_rugpt2/vocab.json',
 './fine_tuned_rugpt2/merges.txt',
 './fine_tuned_rugpt2/added_tokens.json')

In [10]:
from transformers import pipeline

In [11]:
fine_tuned_model = GPT2LMHeadModel.from_pretrained("./fine_tuned_rugpt2")

fine_tuned_tokenizer = GPT2Tokenizer.from_pretrained("./fine_tuned_rugpt2")

In [24]:
generator = pipeline("text-generation", model=fine_tuned_model, tokenizer=fine_tuned_tokenizer, device=0)

num_samples = 5

result = generator(
    "<|startoftext|>",
    max_length=100,             # Максимальная длина текста
    num_return_sequences=5,    # Сколько сэмплов нужно
    do_sample=True,            # Включение стохастической генерации
    temperature=1.0,           # Регулировка разнообразия
    top_k=50,                  # Оставить только 50 наиболее вероятных токенов
    top_p=0.9                  # Nucleus sampling: использовать слова, сумма вероятностей которых <= 0.9
)

for i, sample in enumerate(result):
    print(f"\nSample {i + 1}:\n{sample['generated_text']}\n")
    print('#' * 120)


Sample 1:
<|startoftext|>В одном из кафе.- Мадам, это же вы вчера продали мне торт "Наполеон"?- Да, а что?- А почему у вас тогда нет ни одного ребенка?!

Журналист спрашивает у кинозвезды, почему она до сих пор не замужем.- Понимаете, я не хочу выходить замуж, чтобы не портить свой гардероб.

- Как ты познакомился с моей женой?- Я проходил мимо и увидел, как она идет по улице, ну я и решил подойти и познакомиться

########################################################################################################################

Sample 2:
<|startoftext|>Об этом уже знают все, кроме тех, кто на это подписался.

- Да вы просто п@рнуха какой-то! - заявила соседка моей жене после того, как я купил у нее пылесос.- Да, конечно, но зато в хорошем качестве!

Как известно, самые лучшие друзья - это враги.Но бывает так, что именно друзья могут стать самыми злейшими врагами.

Приходит муж домой, жена с работы -

###############################################################################